# Cellular Automaton

Port of [Think Complexity chapter 5](http://greenteapress.com/complexity2/html/index.html) by Allen Downey.

In [ ]:
using Pkg        # Only first time
pkg"add Luxor"   # Only first time

using Luxor

Cellular Automaton = discrete space (cells) as input for a calculation in discrete time

## A Trivial Example

0 dimensional CA, inverting its cell at each timestep (2 state values only)

rule:

In [ ]:
function rule0dim(x::Bool)
    !x
end

time evolution:

In [ ]:
function step0dim(x₀::Bool, steps::Int64)
    xs = [x₀]
    for i in 1:steps
        push!(xs, rule0dim(xs[end]))
    end
    xs
end

visualisation:

In [ ]:
res = step0dim(false, 10)
Drawing(50, 120, "out.svg")
for (i, val) in enumerate(res)
    if val
        sethue("grey")
    else
        sethue("lightgrey")
    end
    box(5, 5+i*10, 10, 10, :fill)
end
finish()
preview()

## Wolfram's Experiment

1 dimensional CA with 2 state values, new value of a cell depends only on state of neighbouring cells.

rule can be expressed as an integer

|prev|111|110|101|100|011|010|001|000|
|---|---|---|---|---|---|---|---|
|next|b7|b6|b5|b4|b3|b2|b1|b0|
|rule50|0|0|1|1|0|0|1|0|

b can be converted to an integer

get a rule from an integer:

In [ ]:
function inttorule1dim(val::UInt8)
    digs = BitArray(digits(val, base=2))
    for i in length(digs):7
        push!(digs, false)
    end
    digs
end

Apply rule to a cell knowing its own previous state and the previous state of his left and right neighbour

In [ ]:
function applyrule1dim(rule::BitArray{1}, bits::BitArray{1})
    val = 1 + bits[3] + 2*bits[2] + 4*bits[1]
    rule[val]
end

time evolution:

In [ ]:
function step1dim(x₀::BitArray{1}, rule::BitArray{1}, steps::Int64)
    xs = [x₀]
    len = length(x₀)
    for i in 1:steps
        x = copy(x₀)
        for j in 2:len-1
            x[j] = applyrule1dim(rule, xs[end][j-1:j+1])
        end
        push!(xs, x)
    end
    xs
end

initialisation:

In [ ]:
x₀ = falses(21)
x₀[11] = true
res = step1dim(x₀, inttorule1dim(UInt8(50)), 9);

visualisation:

In [ ]:
function visualize1dim(res, dim)
    width = dim * (length(res[1]) + 1)
    height = dim * (length(res) + 1)
    Drawing(width, height, "out.svg")
    for (i, arr) in enumerate(res)
        for (j, val) in enumerate(arr)
            if val
                sethue("grey")
            else
                sethue("lightgrey")
            end
            box(j*dim, i*dim, dim, dim, :fill)
        end
     end
     finish()
     preview()
end

In [ ]:
visualize1dim(res, 10)

## Classifying CAs

### Class 1

Evolution from any starting condition to the same uniform pattern, eg. rule0

In [ ]:
using Random
x₀ = bitrand(21)
x₀[1] = false; x₀[end] = false;
res = step1dim(x₀, inttorule1dim(UInt8(0)), 1)
visualize1dim(res, 10)

### Class 2

Generation of a simple pattern with nested structure, i.e. a pattern that contains many smaller versions of itself, eg. rule50.

Example that looks like a Sierpinsi triangle (fractal): rule18.

In [ ]:
x₀ = falses(129)
x₀[65] = true
res = step1dim(x₀, inttorule1dim(UInt8(18)), 63);
visualize1dim(res, 4)

### Class 3

CAs that generate randomness, eg. rule30.

In [ ]:
x₀ = falses(201)
x₀[101] = true
res = step1dim(x₀, inttorule1dim(UInt8(30)), 99);
visualize1dim(res, 2.5)

Center column as a sequence of bits, is hard to distinguish from a truly random sequence: pseudo-random number generators (PRNGs).

- regularities can be detected statistically
- a PRNG with finite amount of state will eventually repeat itself (period)
- underlying process is fundamentally deterministic (unlike some physical processes: thermodynamics or quantum mechanics)

This complex behavior is surprising (chaos is often associated with non-linear behavior of continuous time and space processes).

### Class 4

CAs that are Turing complete or universal, which means that they can compute any computable function, eg. rule110.

In [ ]:
x₀ = bitrand(600)
res = step1dim(x₀, inttorule1dim(UInt8(110)), 599);
visualize1dim(res, 0.85)

- After about 100 steps, simple repeating patterns emerge, but there are a number of persistent structures that appear as disturbances. Some are vertical, other are diagonal and are called spaceships.

- Collisions between spaceships yields different results depending on their type and their phase. Some collisions annihilate both ships; other leaves one ship unchanged; still other yield one or more ships of different types.

- The collisions are the basis of computation in a rule110 CA. You can think of spaceships as signales that propagate through space, and collisions as gate that compute logical operations like AND and OR.

## Turing State-Machines

Based on [wikipedia: Turing Machine](https://en.wikipedia.org/wiki/Turing_machine).

A Turing machine is a mathematical model of computation that defines an abstract machine, which manipulates symbols on a tape according to a table of rules. Despite the model's simplicity, given any computer algorithm, a Turing machine capable of simulating that algorithm's logic can be constructed.

- A tape divided into cells, one next to the other. Each cell contains a symbol from some finite alphabet. The alphabet contains a special blank symbol (here written as '0') and one or more other symbols. The tape is assumed to be arbitrarily extendable to the left and to the right, i.e., the Turing machine is always supplied with as much tape as it needs for its computation. Cells that have not been written before are assumed to be filled with the blank symbol. In some models the tape has a left end marked with a special symbol; the tape extends or is indefinitely extensible to the right.
- A head that can read and write symbols on the tape and move the tape left and right one (and only one) cell at a time. In some models the head moves and the tape is stationary.
- A state register that stores the state of the Turing machine, one of finitely many. Among these is the special start state with which the state register is initialized. These states, writes Turing, replace the "state of mind" a person performing computations would ordinarily be in.
- A finite table of instructions that, given the state the machine is currently in and the symbol it is reading on the tape (symbol currently under the head), tells the machine to do the following in sequence:
    - Erase or write a symbol.
    - Move the head ( 'L' for one step left or 'R' for one step right or 'N' for staying in the same place).
    - Assume the same or a new state as prescribed.

Table of rules:

| Tape Symbol | State A   | State B   | State C   |
|:-----------:|-----------|-----------|-----------|
| 0           | 1 - R - B | 1 - L - A | 1 - L - B |
| 1           | 1 - L - C | 1 - R - B | 1 - R - H |


In [ ]:
function applyrulebusybeaver(state, read)
    if state == 'A' && read == 0
        return 1, 'R', 'B'
    elseif state == 'A' && read == 1
        return 1, 'L', 'C'
    elseif state == 'B' && read == 0
        return 1, 'L', 'A'
    elseif state == 'B' && read == 1
        return 1, 'R', 'B'
    elseif state == 'C' && read == 0
        return 1, 'L', 'B'
    elseif state == 'C' && read == 1
        return 1, 'R', 'H'
    end
end

struct to represent the Turing State-Machine:

In [ ]:
mutable struct Turing
    tape :: Array{Int64}
    position :: Int64
    state :: Char
end

In [ ]:
function Base.show(io::IO, turing::Turing)
    print(io, turing.position, " - ", turing.state, ": ", turing.tape)
end

implementation of a step:

In [ ]:
function stepturing(turing, applyrule)
    if turing.state == 'H'
        error("Machine has stopped!")
    end
    read = turing.tape[turing.position]
    (write, dir, turing.state) = applyrule(turing.state, read)
    turing.tape[turing.position] = write
    if dir == 'L'
        if turing.position == length(turing.tape)
            push!(turing.tape, false)
        end
        turing.position += 1
    else
        if turing.position == 1
            pushfirst!(turing.tape, false)
        else
            turing.position -= 1
        end
    end
    nothing
end

output:

In [ ]:
turing = Turing(zeros(Int64, 11), 6, 'A')
println(turing)
try
    while true
        stepturing(turing, applyrulebusybeaver)
        println(turing)
    end
catch
end